In [15]:
import torch
import torchvision.datasets as dsets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import torch.nn as nn
import matplotlib.pyplot as plt
import random
USE_CUDA = torch.cuda.is_available() 
device = torch.device("cuda" if USE_CUDA else "cpu") 
print("device", device)

device cpu


In [16]:
class bnn(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden_size = 10
        self.relu = nn.ReLU()
        
        self.w1 = nn.Parameter(
            torch.zeros(2, self.hidden_size, dtype = torch.float32, requires_grad=True)
        )
        self.b = nn.Parameter(
            torch.zeros(self.hidden_size, dtype = torch.float32,requires_grad=True)
        )
        self.w2 = nn.Parameter(
            torch.zeros(self.hidden_size, 1, dtype = torch.float32, requires_grad=True)
        )
        
        
    def rep(self, mu):
        return mu + torch.randn_like(mu)

    def forward(self, x, batch_size):
        b = self.rep(self.b)
        w1 = self.rep(self.w1)
        w2 = self.rep(self.w2)
        b = b.expand(batch_size, self.hidden_size)
        x = torch.matmul(x, w1) + b
        x = self.relu(x)
        x = torch.matmul(x, w2)
        return x, w1, w2, b
    
    def kldloss(self, w1, w2, b):
        sum1 = torch.sum(torch.square(self.w1 - w1))
        sum2 = torch.sum(torch.square(self.w2 - w2))
        sum3 = torch.sum(torch.square(self.b - b))
        return sum1 + sum2 + sum3
    
mybnn = bnn()



In [17]:
tmp_data = torch.tensor([[-1., 3., 5], [-2, 2, 5], [-0.5, 2, 4], 
                         [-0.5, 1.5, 3], [0.5, 1, 1], [1, 1, -1],
                         [2, 2, -3], [3, 2, -4], [4, 4, -5],
                         [5, 4, -4], [5, 5, -3], [6, 4, -2],
                         [7, 4, -1], [8, 3, 0], [8, 4, 1],
                         [9, 3, 3], [9, 1, 5], [10, 2, 6]])
batch_size = 3
hidden_size = 10

w1 = torch.zeros(2, hidden_size)
b = torch.zeros(hidden_size)
w2 = torch.zeros(hidden_size, 1)



In [18]:
train_dataloader = DataLoader(tmp_data, batch_size=3, shuffle=True)
train_data = next(iter(train_dataloader))
newdata = torch.split(train_data, 2, dim = 1)

In [19]:
newdata

(tensor([[10.,  2.],
         [ 9.,  1.],
         [ 5.,  5.]]),
 tensor([[ 6.],
         [ 5.],
         [-3.]]))

In [20]:
output = mybnn(newdata[0],batch_size)
likelihood = torch.sum(torch.square(newdata[1] - output[0]))
loss = likelihood + mybnn.kldloss(w1,w2,b)

tensor([-0.6540,  0.7314, -0.0494, -0.4802,  1.2157, -1.3035, -0.8805,  0.0429,
         0.5185,  0.1716], grad_fn=<AddBackward0>)
tensor([[ 1.4855, -0.3585,  0.1545,  1.8445,  0.6466,  0.2432, -1.2480, -0.9764,
         -0.3140,  1.1787],
        [ 0.7085,  0.2284,  0.3278,  0.5273, -0.1377,  0.3444, -0.3726,  0.1899,
          1.6828, -0.6465]], grad_fn=<AddBackward0>)
tensor([[-0.3299],
        [ 0.3854],
        [-1.5208],
        [-0.3533],
        [ 1.0492],
        [-0.7111],
        [ 0.5599],
        [ 1.0655],
        [ 0.1236],
        [ 1.5977]], grad_fn=<AddBackward0>)


In [21]:
optimizer = torch.optim.SGD(mybnn.parameters(), lr=0.01)


In [22]:
optimizer.zero_grad()
loss.backward()
optimizer.step()
print(loss)

tensor(28.2256, grad_fn=<AddBackward0>)


In [23]:
mybnn.w1

Parameter containing:
tensor([[  44.9442,   -0.8978,  207.1793,   48.1342, -142.9255,   96.8720,
            0.0000,    0.0000,   -6.3796, -217.6474],
        [   7.1222,   -0.8978,   32.8311,    7.6277,  -22.6490,   15.3510,
            0.0000,    0.0000,   -1.5063,  -34.4899]], requires_grad=True)

In [24]:
mybnn.b

Parameter containing:
tensor([  4.8815,  -0.1796,  22.5021,   5.2279, -15.5234,  10.5215,   0.0000,
          0.0000,  -0.6668, -23.6391], requires_grad=True)

In [25]:
mybnn.w2

Parameter containing:
tensor([[-2.0799e+02],
        [-3.7866e-02],
        [-2.7396e+01],
        [-2.5556e+02],
        [-1.0310e+02],
        [-2.1282e+01],
        [ 0.0000e+00],
        [ 0.0000e+00],
        [-7.0985e+00],
        [-1.4915e+02]], requires_grad=True)